# Scratch notebook for experimenting with ridge trace algorithm

## Basic plan

0. We use a map of surface brightness, which we will call $B$
1. We start at some point on the emission arc that we want to trace. For instance, the local maximum brightness
2. We find the direction $\hat{s}$ where the curvature 

In [1]:
import sys

In [2]:
sys.version_info

sys.version_info(major=3, minor=12, micro=1, releaselevel='final', serial=0)